# Importação de funções e montagem do diretório

In [ ]:
!pip install opencv-python==4.5.5.64

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
cv2.__version__

'4.5.5'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
dir_fernanda = '/content/gdrive/MyDrive/UFAL/PDI/Projeto AB2 - PDI'
dir_neto = '/content/gdrive/MyDrive/Compartilhado/Projeto AB2 - PDI'

path_dir = dir_fernanda

In [ ]:
df = pd.read_csv(f"{path_dir}/train.csv")
df = df.fillna(0)

# Funções auxiliares

In [ ]:
def get_rgb(img_name, dir=f"{path_dir}/img", plot=False):
  img = cv2.imread(f"{dir}/{img_name}")
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  if plot:
    plt.imshow(img)
    plt.show()

  return img

In [ ]:
def middle_bottle(img, offset=15, plot=False):
  linha, col, _ = img.shape

  col_inicio = int((col / 3) - offset)
  col_fim = int(col - (col / 3) + offset)
  
  result = img[:,col_inicio:col_fim,:]

  if plot:
    plt.imshow(result)
    plt.show()

  return result

In [ ]:
def bottle_label(img, offset=15, plot=False):
    linha, col, _ = img.shape

    linha_inicio = int(linha / 2)
    linha_fim = linha - offset

    result = img[linha_inicio:linha_fim,:,:]

    if plot:
        plt.imshow(result)
        plt.show()

    return result

# Funções de detecção de defeitos

In [ ]:
def content_high(image, plot=False, lim=2000):
  '''
    image: imagem da garrafa do meio em RGB
    plot: visualização da imagem com classificação
    lim: área mínima para região com refrigerante
  
    return: 0 caso nível do conteúdo seja normal e 1 caso esteja mais alto que o normal
  '''
  img_bw = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

  linha, col= img_bw.shape
  fim = linha // 2
  half1 = img_bw[:fim,:]

  inicio = fim // 2
  content = half1[inicio:,:]

  _, binary_image = cv2.threshold(content, 150, 255, cv2.THRESH_BINARY_INV)
  black_area = np.count_nonzero(binary_image)

  content_high = False
  if black_area > lim:
    content_high = True

  if plot:
    plt.imshow(binary_image, cmap="gray")
    plt.title(f"Content High: {content_high}")
    plt.show()

  return int(content_high)

In [ ]:
def content_low(image, plot=False, lim=300):
  '''
    image: imagem da garrafa do meio em RGB
    plot: visualização da imagem com classificação
    lim: área mínima para região com refrigerante
  
    return: 0 caso nível do conteúdo seja normal e 1 caso esteja mais baixo que o normal
    OBS: não funciona corretamente em garrafas amassadas
    OBS: em caso de não existir garrafa, o valor será 1
  '''
  img_bw = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

  linha, col= img_bw.shape
  fim = linha // 2
  half1 = img_bw[:fim,:]

  inicio = fim // 2
  content = half1[inicio:,:]

  _, binary_image = cv2.threshold(content, 150, 255, cv2.THRESH_BINARY_INV)
  black_area = np.count_nonzero(binary_image)

  content_low = False
  if black_area < lim:
    content_low = True

  if plot:
    plt.imshow(binary_image, cmap="gray")
    plt.title(f"Content Low: {content_low}")
    plt.show()

  return int(content_low)

In [ ]:
def cover_none(image, plot=False, lim=500):
  '''
    image: imagem da garrafa do meio em RGB
    plot: visualização da imagem com classificação
    lim: área mínima para região vermelha
  
    return: 0 caso garrafa tenha tampa e 1 caso não exista
    OBS: em caso de não existir garrafa, o valor será 1
  '''
  hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

  lower_red = np.array([0, 100, 100])   
  upper_red = np.array([10, 255, 255])   

  red_mask = cv2.inRange(hsv_image, lower_red, upper_red)
  red_image = cv2.bitwise_and(image, image, mask=red_mask)

  linha, col, _ = image.shape
  fim = linha // 2
  red_half = red_image[:fim,:,:]

  red_area = np.count_nonzero(red_half)
  
  cover_none = True
  if red_area > lim:
    cover_none = False

  if plot:
    plt.imshow(image)
    plt.title(f"Cover none: {cover_none}")
    plt.show()

  return int(cover_none)

In [ ]:
def bottle_smashed(garrafa, plot=False, threshold=30, lim_min_width=80, lim_max_width=135):
    '''
      garrafa: imagem da garrafa em RGB
      plot: visualização da imagem com marcações
      threshold: limite para detecção de bordas
      min_width: largura mínima para considerar o líquido como amassado
      
      return: True se o líquido estiver amassado, False caso contrário
    '''

    hsv_image = cv2.cvtColor(garrafa, cv2.COLOR_RGB2HSV)
    lower_black = np.array([0, 0, 0])
    upper_black = np.array([255, 255, 150])
    mask = cv2.inRange(hsv_image, lower_black, upper_black)
    liquid = cv2.bitwise_and(garrafa, garrafa, mask=mask)

    gray = cv2.cvtColor(liquid, cv2.COLOR_RGB2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, threshold, threshold*2)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    max_width = 0

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w > max_width:
            max_width = w

    bottle_smashed = max_width > lim_min_width and max_width < lim_max_width

    if plot:
        contour_img = np.copy(garrafa)
        cv2.drawContours(contour_img, contours, -1, (0, 255, 0), 2)

        plt.title(f"Bottle smashed: {bottle_smashed}")
        plt.imshow(contour_img)
        plt.show()

    return int(bottle_smashed)

In [ ]:
def label_white(image, plot=False, lim=20000, lim_sup=34000):
    '''
      image: imagem da garrafa em RGB
      plot: visualização da imagem com classificação
      lim: área mínima para região branca
      
      return: 1 caso o rótulo esteja branco, 0 caso contrário
    '''
    image = bottle_label(image)

    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

    lower_white = np.array([12, 15, 200])
    upper_white = np.array([255, 255, 255])

    white_mask = cv2.inRange(hsv_image, lower_white, upper_white)
    white_image = cv2.bitwise_and(image, image, mask=white_mask)
    white_area = np.count_nonzero(white_image)

    label_white = False
    if white_area > lim and white_area < lim_sup:
        label_white = True

    if plot:
        plt.imshow(image)
        plt.title(f"Label white: {label_white}")
        plt.show()

    return int(label_white)

In [ ]:
def label_misplaced(garrafa, plot=False, divisao=3, lim=4):
  '''
    garrafa: imagem da garrafa do meio em RGB
    plot: visualização da imagem com classificação
    divisao: valor para selecionar a região central da garrafa
    lim: desnível mínimo para linha com o rótulo
  
    return: 0 caso rótulo esteja normal e 1 caso esteja desnivelado
  '''

  lin, col, _ = garrafa.shape
  inicio = lin // 2

  final = col 
  metade_garrafa = garrafa[inicio:inicio+final,:,:]

  hsv_image = cv2.cvtColor(metade_garrafa, cv2.COLOR_RGB2HSV)

  lower_red = np.array([0, 100, 100])   
  upper_red = np.array([10, 255, 255])   

  filtro_vermelho = cv2.inRange(hsv_image, lower_red, upper_red)
  img_vermelha = cv2.bitwise_and(metade_garrafa, metade_garrafa, mask=filtro_vermelho)
  imagem_filtrada = cv2.medianBlur(img_vermelha, 5) 

  colunas = imagem_filtrada.shape[1]
  inicio_meio = colunas // divisao
  fim_meio = colunas - inicio_meio
  meio_garrafa = imagem_filtrada[:,inicio_meio:fim_meio,:]

  desnivel = 0

  for i in range(meio_garrafa.shape[0]):
    pixel_vermelho = np.nonzero(meio_garrafa[i,:,0])[0]
    if np.isin([0,meio_garrafa.shape[1]-1], pixel_vermelho)[-1]:
      break

    if len(pixel_vermelho)> 0:
      desnivel += 1
  
  if desnivel >= lim:
    label_misplaced = True
  else:
    label_misplaced = False

  if plot:
    plt.title(f"Label misplaced: {label_misplaced}")
    plt.imshow(meio_garrafa)
    plt.show()

  return int(label_misplaced)

In [ ]:
def label_none(image, plot=False, lim=15000):
    '''
      image: imagem da garrafa em RGB
      plot: visualização da imagem com classificação
      lim: área mínima para região preta (líquido)
      
      return: 1 caso não exista rótulo, 0 caso contrário
    '''

    image = bottle_label(image)

    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

    lower_black = np.array([0, 0, 0])
    upper_black = np.array([180, 255, 40])

    black_mask = cv2.inRange(hsv_image, lower_black, upper_black)
    black_image = cv2.bitwise_and(image, image, mask=black_mask)
    black_area = np.count_nonzero(black_image)

    label_none = False
    if black_area > lim:
        label_none = True

    if plot:
        plt.imshow(image)
        plt.title(f"Label none: {label_none}")
        plt.show()

    return int(label_none)

In [ ]:
def bottle_none(image, plot=False, lim=47500):
    '''
      image: imagem da garrafa em RGB
      plot: visualização da imagem com classificação
      lim: área mínima para região branca
      
      return: 0 caso garrafa esteja presente e 1 caso contrário
    '''
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

    lower_white = np.array([10, 50, 50])
    upper_white = np.array([150, 255, 220])

    white_mask = cv2.inRange(hsv_image, lower_white, upper_white)
    white_image = cv2.bitwise_and(image, image, mask=white_mask)
    white_area = np.count_nonzero(white_image)

    bottle_none = False
    if white_area > lim:
        bottle_none = True

    if plot:
        plt.imshow(image)
        plt.title(f"Bottle none: {bottle_none}")
        plt.show()

    return int(bottle_none)

# Uso das funções

In [ ]:
def detect_defects(image):
  bottle =  middle_bottle(image)

  detected = {}

  detected['content_high'] = content_high(bottle)
  detected['content_low'] = content_low(bottle)
  detected['cover_none'] = cover_none(bottle)
  detected['bottle_smashed'] = bottle_smashed(bottle)
  detected['label_white'] = label_white(bottle)
  detected['label_misplaced'] = label_misplaced(bottle)
  detected['label_none'] = label_none(bottle)
  detected['bottle_none'] = bottle_none(bottle)

  return np.array(list(detected.values()))

In [ ]:
'''

  Insira aqui a chamada da função principal

'''